In [1]:
import numpy as np
import re
import time
import math

In [2]:
with open("input_17.txt", "r") as fh:
    content_registers, content_program = fh.read().split("\n\n")

In [3]:
A, B, C = [int(r) for r in re.findall(r"(\d+)", content_registers)]
program = [int(p) for p in re.findall(r"(\d)", content_program)]
program

[2, 4, 1, 1, 7, 5, 0, 3, 1, 4, 4, 4, 5, 5, 3, 0]

In [4]:
# 17a
t0 = time.time()
def combo(operand, A, B, C):
    if 0 <= operand <= 3:
        return operand
    match operand:
        case 4: return A
        case 5: return B
        case 6: return C
    print("Error: this should not be reached")


def interpret(program, A_input, B_input, C_input, with_print=False):
    A = A_input
    B = B_input
    C = C_input
    pointer = 0
    output = []
    
    while pointer < len(program):
        opcode, operand = program[pointer:pointer+2]
    
        match opcode:
            case 0:
                # adv
                A = int(A / (2**combo(operand, A, B, C))) # 17b step 4: always: A / 8
            case 1:
                # bxl
                B ^= operand # 17b step 2 and step 5: B LSB flipped (right most bit)
            case 2:
                # bst
                B = combo(operand, A, B, C) % 8 # 17b step 1: B = A % 8 (i.e., B in range(000, 111))
            case 3:
                # jnz
                if A != 0:
                    pointer = operand - 2
            case 4:
                # bxc
                B ^= C # 17b step 6
            case 5:
                # out
                out = combo(operand, A, B, C) % 8 # 17b step 7: always B % 8
                # Needed for analysis of 17b
                if with_print:
                    print("out", out, bin(out))
                output.append(out)
            case 6:
                # bdv
                B = int(A / (2**combo(operand, A, B, C)))
            case 7:
                # cdv
                C = int(A / (2**combo(operand, A, B, C))) # 17b step 3: C = A / 2**B (i.e., A / range(1, 2**7=128)


        # Needed for analysis of 17b
        if with_print:
            print((opcode, operand))
            print("A", A, str(bin(A))[2:], len(str(bin(A))) - 2)
            print("B", B, str(bin(B))[2:], len(str(bin(B))) - 2)
            print("C", C, str(bin(C))[2:], len(str(bin(C))) - 2)
        
        pointer += 2
    return output

print("17a:", ",".join([str(o) for o in interpret(program, A, B, C)]), f"in {time.time()-t0} sec.")

17a: 6,1,6,4,2,4,7,3,5 in 0.00035190582275390625 sec.


In [5]:
# 17b
t0 = time.time()
# Absolute Min and max values determined based on length of output. Binary digits of A reduce by 3 every iteration
A_min = 2**((len(program) - 1)*3)
A_max = 2**((len(program))*3) - 1
max_exp = int(math.log2(A_min))

# Solve by searching output from right to left. Search with decreasing step-sizes.
A0 = A_min
res = interpret(program, A0, 0, 0, with_print=False)
exp = max_exp
compare_element = -1
while res != program:
    exp -= 2
    count = 1
    A = A0
    # Each loop tries to find next output from the right by searching a binary level of magnitude more accurate.
    # When a new output digit is found, the exponent will be decreased, and the minimum A to start searching increased.
    while count < 10000:
        res = interpret(program, A, 0, 0, with_print=False)
        if res[compare_element:] == program[compare_element:]:
            # Update A0 and add extra element to compare in the future iterations
            A0 = A
            compare_element -= 1
            break
        count += 1
        A += 2**exp
    print(f"Searching with step-size 2**{exp} --> New starting A0={A0}")
print("17b:", A, f"in {time.time()-t0} sec.")

Searching with step-size 2**43 --> New starting A0=175921860444160
Searching with step-size 2**41 --> New starting A0=202310139510784
Searching with step-size 2**39 --> New starting A0=202310139510784
Searching with step-size 2**37 --> New starting A0=202310139510784
Searching with step-size 2**35 --> New starting A0=202310139510784
Searching with step-size 2**33 --> New starting A0=202404628791296
Searching with step-size 2**31 --> New starting A0=202406776274944
Searching with step-size 2**29 --> New starting A0=202406776274944
Searching with step-size 2**27 --> New starting A0=202975054135296
Searching with step-size 2**25 --> New starting A0=202975054135296
Searching with step-size 2**23 --> New starting A0=202975171575808
Searching with step-size 2**21 --> New starting A0=202975182061568
Searching with step-size 2**19 --> New starting A0=202975183634432
Searching with step-size 2**17 --> New starting A0=202975183634432
Searching with step-size 2**15 --> New starting A0=20297518363